In [1]:
import group_data as gd
import flylib
import numpy as np

In [2]:
%pylab inline
%config InlineBackend.figure_format = 'svg'

Populating the interactive namespace from numpy and matplotlib


### Stripe fixation 

In [5]:
#### construct an interpolated matrix of cross correlation data
xcor = list()
flynums = list()
sig_types = list()
muscle_names = list()
swarm_names = list()

for swarm_name,swarm in gd.swarms.items():
    for fly in swarm.flies:
        try:
            pkldata = fly.get_pkl_data('nnls_fits_no_bk_dF_F_corr_data.cpkl')
            x = pkldata['times']
            for sig_type_key in ['amp', 'freq', 'auto']:
                for mkey in pkldata[sig_type_key].keys():
                    y = pkldata[sig_type_key][mkey]
                    from scipy import interpolate
                    tck = interpolate.splrep(x, y, s=0)
                    xnew = np.arange(-100,100,0.01)
                    ynew = interpolate.splev(xnew, tck, der=0)
                    xcor.append(ynew)
                    muscle_names.append(mkey)
                    sig_types.append(sig_type_key)
                    swarm_names.append(swarm_name)
                    flynums.append(fly.fly_num)
        except IOError:
            print 'no input file'

xcor = np.array(xcor)
flynums = np.array(flynums)
sig_types = np.array(sig_types)
muscle_names = np.array(muscle_names)
swarm_names = np.array(swarm_names)

no input file


In [9]:
import os
os.remove('extracted_signal_correlations.hdf5')
import h5py
sig_cor = h5py.File('extracted_signal_correlations.hdf5')
sig_cor['xcor'] = xcor
sig_cor['fly_nums'] = flynums
sig_cor['sig_types'] = sig_types
sig_cor['muscle_names'] = muscle_names
sig_cor['swarm_names'] = swarm_names
sig_cor.flush()
sig_cor.close()

### Motion stimuli trial matrix

In [3]:
trial_numbers = list()
trial_names = list()
fly_numbers = list()
signal_units = list()
signal_data = list()
signal_names = list()
swarm_names = list()
for swarm_name,swarm in gd.swarms.items():
    for fly in swarm.flies:
        try:
            sigs = fly.get_pkl_data('nnls_fits_no_bk_dF_F_sig_mtrx.cpkl')
            for i,trial in enumerate(sigs['StimCond']):
                for key in ['StimCond','wb_frequency',
                            'Ph1','Ph2','Xpos','Ypos',
                            'b1','b2','b3',
                            'i1','i2',
                            'iii1','iii3','iii24',
                            'hg1','hg2','hg3','hg4',
                            'tpd','tpv','ttm']:
                    try:
                        signal_data.append(np.array(sigs[key][i]))
                        trial_names.append(gd.decode[np.around(np.mean(trial[trial>0.1]))])
                        trial_numbers.append(i)
                        fly_numbers.append(fly.fly_num)
                        signal_names.append(key)
                        swarm_names.append(swarm_name)
                    except KeyError:
                        pass
        except IOError:
            print fly.fly_num
            print swarm_name
            print 'missing file'
                #print 'missing: %s'%key
        except EOFError:
            print 'eof error' + str(fly.fly_num)
            
trial_numbers = np.array(trial_numbers)
trial_names = np.array(trial_names)
fly_numbers = np.array(fly_numbers)
signal_units = np.array(signal_units)
signal_data = np.array(signal_data)
signal_names = np.array(signal_names)
swarm_names = np.array(swarm_names)

eof error313


In [4]:
import os
try:
    os.remove('extracted_step_trans_yaw.hdf5')
except OSError as err:
    print err 
import h5py
sig_file = h5py.File('extracted_step_trans_yaw.hdf5')
sig_file['trial_numbers'] = trial_numbers
sig_file['trial_names'] = trial_names
sig_file['fly_numbers'] = fly_numbers
sig_file['signal_units'] = signal_units
sig_file['signal_data'] = signal_data
sig_file['signal_names'] = signal_names
sig_file['swarm_names'] = swarm_names

In [5]:
sig_file.flush()
sig_file.close()